<a href="https://colab.research.google.com/github/sunyingjian/AI-in-well-logging/blob/master/Conv1d_PE%E5%9B%9E%E5%BD%92_regression_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sunyingjian/numpy-.git

Cloning into 'numpy-'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 406 (delta 0), reused 0 (delta 0), pack-reused 403
Receiving objects: 100% (406/406), 202.19 MiB | 33.53 MiB/s, done.
Resolving deltas: 100% (114/114), done.
Checking out files: 100% (204/204), done.


In [2]:
! /opt/bin/nvidia-smi

Sat Jan 23 09:51:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tensorflow import keras
%matplotlib inline
import tensorflow_addons as tfa

In [4]:
data = pd.read_csv('/content/numpy-/TCN data.csv')
data

,Unnamed: 0,Formation,Depth,GR,ILD_log10,DeltaPHI,PHIND,NM_M,RELPOS,Facies,Well Name,PE
0,0,0.076923,0.399818,0.298966,0.458149,0.776042,0.219321,0,1.000000,3,7,0.557385
1,1,0.076923,0.400729,0.302738,0.456157,0.888021,0.231865,0,0.978788,3,7,0.494046
2,2,0.076923,0.401639,0.306417,0.454165,0.903646,0.241224,0,0.956566,3,7,0.430707
3,3,0.076923,0.402550,0.339247,0.452173,0.880208,0.242479,0,0.935354,3,7,0.418039
4,4,0.076923,0.403461,0.285601,0.446860,0.869792,0.246049,0,0.914141,3,7,0.405371
...,...,...,...,...,...,...,...,...,...,...,...,...
3227,3227,0.923077,0.996357,0.155858,0.646070,0.565833,0.129373,1,0.681818,5,0,0.432860
3228,3228,0.923077,0.997268,0.145818,0.650055,0.576589,0.144021,1,0.673737,5,0,0.398277
3229,3229,0.923077,0.998179,0.169829,0.657359,0.594401,0.144021,1,0.665657,5,0,0.378769
3230,3230,0.923077,0.999089,0.177978,0.658023,0.598516,0.138135,1,0.657576,5,0,0.373955


In [5]:
data['Well Name'].value_counts()

1    501
7    471
3    463
2    461
6    449
4    415
0    404
5     68
Name: Well Name, dtype: int64

# 分割数据集

In [6]:
test_data = data.loc[data['Well Name']==1]

In [7]:
test_data = test_data.drop(columns=['Unnamed: 0','Well Name'])

In [8]:
index=data[data['Well Name'].isin([1])].index[0]

In [9]:
training_data = data.drop(index = [i for i in range(1381,1882)])

In [10]:
train_data = training_data.drop(columns=['Well Name','Unnamed: 0'])

In [11]:
train_data =  train_data.reset_index(drop=True)

In [12]:
test_data = test_data.reset_index(drop=True)

In [13]:
train_data

,Formation,Depth,GR,ILD_log10,DeltaPHI,PHIND,NM_M,RELPOS,Facies,PE
0,0.076923,0.399818,0.298966,0.458149,0.776042,0.219321,0,1.000000,3,0.557385
1,0.076923,0.400729,0.302738,0.456157,0.888021,0.231865,0,0.978788,3,0.494046
2,0.076923,0.401639,0.306417,0.454165,0.903646,0.241224,0,0.956566,3,0.430707
3,0.076923,0.402550,0.339247,0.452173,0.880208,0.242479,0,0.935354,3,0.418039
4,0.076923,0.403461,0.285601,0.446860,0.869792,0.246049,0,0.914141,3,0.405371
...,...,...,...,...,...,...,...,...,...,...
2726,0.923077,0.996357,0.155858,0.646070,0.565833,0.129373,1,0.681818,5,0.432860
2727,0.923077,0.997268,0.145818,0.650055,0.576589,0.144021,1,0.673737,5,0.398277
2728,0.923077,0.998179,0.169829,0.657359,0.594401,0.144021,1,0.665657,5,0.378769
2729,0.923077,0.999089,0.177978,0.658023,0.598516,0.138135,1,0.657576,5,0.373955


#分割数据

In [27]:
training_data = training_data.values

In [28]:
seq_length = 32
data_ = []
for i in range(len(train_data)-seq_length):
  if training_data[i,-2]!=training_data[i+seq_length,-2]:
    continue
  data_.append(train_data.iloc[i:i+seq_length])

In [29]:
data_ = np.array([df.values for df in data_])

In [30]:
X = data_[:,:,:9]

In [31]:
Y = data_[:,:,-1]

In [32]:
data_test = []
for i in range(len(test_data)-seq_length):
    data_test.append(test_data.iloc[i:i+seq_length])

In [33]:
data_test = np.array([df.values for df in data_test])

In [34]:
test_x = data_test[:,:,:9]

In [35]:
test_y = data_test[:,:,-1]

In [48]:
X.shape

(2507, 32, 9)

# 建立Conv1D网络

In [37]:
input = tf.keras.Input(shape=(32,9))

In [55]:
x = tf.keras.layers.Conv1D(32,kernel_size=2,padding='same',activation='relu')(input)
x = tf.keras.layers.Dense(64,activation='relu',kernel_regularizer='l2')(x)
x = tf.keras.layers.Dense(1)(x)

In [56]:
model = tf.keras.Model(inputs = input,outputs = x)

In [57]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 9)]           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 32, 32)            608       
_________________________________________________________________
dense_6 (Dense)              (None, 32, 64)            2112      
_________________________________________________________________
dense_7 (Dense)              (None, 32, 1)             65        
Total params: 2,785
Trainable params: 2,785
Non-trainable params: 0
_________________________________________________________________


In [58]:
#学习率衰减
learning_rate=0.01

Lr_change=tf.keras.callbacks.ReduceLROnPlateau('val_mae',patience = 20, factor = 0.5, min_lr=0.0001)
#保存准确率最好的模型
from keras.callbacks import ModelCheckpoint
filepath="best_weight.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_mae', verbose=1, save_best_only=True,mode='min')
Adam=tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=Adam,loss='mse',metrics=['mae'])

In [59]:
history=model.fit( X,Y,batch_size=128,
         epochs=100, 
         callbacks=[Lr_change,checkpoint],
         validation_data=(test_x,test_y))

Epoch 1/100
20/20 [==============================] - 1s 18ms/step - loss: 0.7553 - mae: 0.4717 - val_loss: 0.2099 - val_mae: 0.1517

Epoch 00001: val_mae improved from inf to 0.15173, saving model to best_weight.h5
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.1878 - mae: 0.1063 - val_loss: 0.1504 - val_mae: 0.0797

Epoch 00002: val_mae improved from 0.15173 to 0.07972, saving model to best_weight.h5
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.1418 - mae: 0.0679 - val_loss: 0.1201 - val_mae: 0.0682

Epoch 00003: val_mae improved from 0.07972 to 0.06824, saving model to best_weight.h5
Epoch 4/100
20/20 [==============================] - 0s 5ms/step - loss: 0.1145 - mae: 0.0621 - val_loss: 0.0973 - val_mae: 0.0561

Epoch 00004: val_mae improved from 0.06824 to 0.05610, saving model to best_weight.h5
Epoch 5/100
20/20 [==============================] - 0s 5ms/step - loss: 0.0942 - mae: 0.0584 - val_loss: 0.0799 - val_mae: 0.0503


In [60]:
import sklearn
from sklearn import metrics
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.metrics import r2_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [61]:
def help(model,test_x,test_y):
  y_pred = model.predict(test_x)
  y_pred1 = y_pred[1,:16,:]
  y_pred_2 = y_pred[:,15,:]
  y_pred_3 = y_pred[-1,16:,:]
  y_pred_true = np.append(y_pred1,y_pred_2)
  y_pred_true = np.append(y_pred_true,y_pred_3)
  y_test_1 = test_y[1,:16]
  y_test_2 = test_y[:,15]
  y_test_3 = test_y[-1,16:]
  y_test_true = np.append(y_test_1,y_test_2)
  y_test_true = np.append(y_test_true,y_test_3)
  print(f'MAE={metrics.mean_absolute_error(y_test_true,y_pred_true)}')
  print(f'可决系数R2:{r2_score(y_test_true,y_pred_true)}')

In [62]:
help(model,test_x,test_y)

MAE=0.03717168405799494
可决系数R2:0.6156210700219951
